<a href="https://colab.research.google.com/github/ViniciusFebasse/Preco-das-Casas/blob/main/Pre%C3%A7o_das_Casas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [630]:
import pandas as pd
import numpy as np

In [631]:
df_train = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Preço das casas/train.csv")

In [632]:
df_train.head(2)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,...,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,...,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,...,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500


In [633]:
df_train.shape

(1460, 81)

In [634]:
df_test = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Preço das casas/test.csv")

In [635]:
df_test.head(2)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Feedr,Norm,1Fam,1Story,5,6,1961,1961,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,CBlock,TA,TA,No,Rec,468.0,LwQ,144.0,270.0,882.0,GasA,TA,Y,SBrkr,896,0,0,896,0.0,0.0,1,0,2,1,TA,5,Typ,0,NaN,Attchd,1961.0,Unf,1.0,730.0,TA,TA,Y,140,0,0,0,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,NAmes,Norm,Norm,1Fam,1Story,6,6,1958,1958,Hip,CompShg,Wd Sdng,Wd Sdng,BrkFace,108.0,TA,TA,CBlock,TA,TA,No,ALQ,923.0,Unf,0.0,406.0,1329.0,GasA,TA,Y,SBrkr,1329,0,0,1329,0.0,0.0,1,1,3,1,Gd,6,Typ,0,NaN,Attchd,1958.0,Unf,1.0,312.0,TA,TA,Y,393,36,0,0,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal


In [636]:
df_test.shape

(1459, 80)

In [637]:
df_sample = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Preço das casas/sample_submission.csv")

In [638]:
df_sample.head(2)

,Id,SalePrice
0,1461,169277.052498
1,1462,187758.393989


In [639]:
df_sample.shape

(1459, 2)

In [640]:
# Caso se queira usar a função com o fit_transform passe o parâmetro como 0 e se for como transform passe o parâmetro 1
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, OneHotEncoder
label = LabelEncoder()
minmax = MinMaxScaler()
onehot = OneHotEncoder(sparse=False, drop='first', handle_unknown='ignore')

def preprocessamento(dataframe, fit_trans):
  
  data = None

  # Exclusão do Id
  dataframe.drop('Id', axis=1, inplace=True)

  # Exclusão de colunas com poucos registros
  dataframe.drop('Alley', axis=1, inplace=True)
  dataframe.drop('FireplaceQu', axis=1, inplace=True)
  dataframe.drop('PoolQC', axis=1, inplace=True)
  dataframe.drop('MiscFeature', axis=1, inplace=True)
  dataframe.drop('Fence', axis=1, inplace=True)

  # Exclusão de registros faltantes
  dataframe.dropna(axis=0, how='any', inplace=True)

  # Separação de features e classes
  if fit_trans == 0:
    X = dataframe.drop('SalePrice', axis=1)
    y = dataframe['SalePrice']

  else:
    X = dataframe
  
  # Separação entre dados categóricos e numéricos
  X_cat = X.select_dtypes(include='object')
  X_num = X.select_dtypes(exclude='object')

  # Codificação das features categóricas
  #fit_trans if fit_trans == 0 else transform
  if fit_trans == 0:
    X_cat = onehot.fit_transform(X_cat)
  else:
    X_cat = onehot.transform(X_cat)

  # Escalonamento de todas as variáveis
  if fit_trans == 0:
    X_num = minmax.fit_transform(X_num)
  else:
    X_num = minmax.transform(X_num)

  # Definindo o X completo
  X_comp = np.concatenate((X_cat, X_num), axis=1)

  # Separação em conjuntos de treino e teste
  if fit_trans == 0:
    X_treino, X_teste, y_treino, y_teste = train_test_split(X_comp, y)
  else:
    X_treino, X_teste, y_treino, y_teste = None, None, None, None

  # Retorno dos resultados
  return X_treino, X_teste, y_treino, y_teste, X_comp

X_train, X_test, y_train, y_test, _ = preprocessamento(df_train, 0)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(820, 221) (274, 221) (820,) (274,)


Criação do Modelo de Machine Learning

In [641]:
from sklearn.neural_network import MLPRegressor

neural = MLPRegressor(max_iter=10000, solver='adam', activation='relu', hidden_layer_sizes=(100, 100, 100), tol=0.0001, random_state=0)

neural = MLPRegressor(max_iter=10000, solver='adam', activation='relu', hidden_layer_sizes=(100, 100, 100), tol=0.0001)
-> 87.02%

In [642]:
neural.fit(X_train, y_train)

MLPRegressor(hidden_layer_sizes=(100, 100, 100), max_iter=10000, random_state=0)

In [643]:
predicao = neural.predict(X_test)

Métricas

In [644]:
escore = neural.score(X_test, y_test)

In [645]:
print(f'{np.around(escore * 100, 2)}%')

81.96%


In [646]:
from sklearn.metrics import mean_absolute_error

mae = mean_absolute_error(y_test, predicao)
print(f'$ {np.around(mae, 2)}')

$ 18890.2


Previsão dos dados novos

In [647]:
df_test = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Preço das casas/test.csv")

In [648]:
df_test.head(2)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Feedr,Norm,1Fam,1Story,5,6,1961,1961,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,CBlock,TA,TA,No,Rec,468.0,LwQ,144.0,270.0,882.0,GasA,TA,Y,SBrkr,896,0,0,896,0.0,0.0,1,0,2,1,TA,5,Typ,0,NaN,Attchd,1961.0,Unf,1.0,730.0,TA,TA,Y,140,0,0,0,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,NAmes,Norm,Norm,1Fam,1Story,6,6,1958,1958,Hip,CompShg,Wd Sdng,Wd Sdng,BrkFace,108.0,TA,TA,CBlock,TA,TA,No,ALQ,923.0,Unf,0.0,406.0,1329.0,GasA,TA,Y,SBrkr,1329,0,0,1329,0.0,0.0,1,1,3,1,Gd,6,Typ,0,NaN,Attchd,1958.0,Unf,1.0,312.0,TA,TA,Y,393,36,0,0,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal


In [649]:
df_test.drop(df_test[df_test.MSZoning == 'RH'].index, axis=0, inplace=True)
df_test.drop(df_test[df_test.MSZoning == 'RL'].index, axis=0, inplace=True)
df_test.drop(df_test[df_test.MSZoning == 'RM'].index, axis=0, inplace=True)
df_test.drop(df_test[df_test.MSZoning == 'FV'].index, axis=0, inplace=True)
#df_test.drop(df_test[df_test.MSZoning == 'C (all)'].index, axis=0, inplace=True)

In [650]:
df_test.head(2)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
98,1559,70,C (all),NaN,6449,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,SWISU,Norm,Norm,1Fam,2Story,4,1,1907,1950,Gambrel,CompShg,Wd Sdng,Stucco,None,0.0,TA,TA,CBlock,TA,TA,No,Rec,73.0,Unf,0.0,634.0,707.0,GasW,TA,N,SBrkr,942,942,0,1884,0.0,0.0,1,1,4,1,TA,7,Typ,0,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,N,0,0,239,0,0,0,NaN,NaN,NaN,0,3,2010,WD,Abnorml
142,1603,30,C (all),66.0,8712,Grvl,NaN,Reg,Lvl,AllPub,Corner,Gtl,IDOTRR,Norm,Norm,1Fam,1Story,4,7,1896,1950,Hip,CompShg,Wd Sdng,Wd Sdng,None,0.0,Fa,Fa,CBlock,TA,TA,No,Unf,0.0,Unf,0.0,756.0,756.0,GasA,Gd,Y,SBrkr,952,0,0,952,0.0,0.0,1,0,3,1,TA,5,Typ,0,NaN,Detchd,1896.0,RFn,1.0,330.0,TA,TA,N,0,0,265,0,0,0,NaN,NaN,NaN,0,6,2010,WD,Alloca


In [651]:
_, _, _, _, X_prever = preprocessamento(df_test, 1)

In [652]:
X_prever.shape

(5, 221)

In [653]:
predicao_nova = neural.predict(X_prever)
np.around(predicao_nova, 2)

array([ 94441.45,  94710.09,  59086.27, 140328.65, 131694.55])